# Data Processing

Objective:

- Clean location, salary, titles 
- Make sure data size is large enough
- Clean JD 
    - Remove stop words
    - basic tokenization
    - get most frequently used words
    - create bigrams, or ngrams, or bag or words
    - count vectorization

In [1]:
import time
import nltk
from nltk.corpus import stopwords

from nltk import word_tokenize, FreqDist
from nltk.corpus import stopwords
from nltk.collocations import *
import string, re

from gensim.models import Word2Vec

from sklearn.manifold import TSNE
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
np.random.seed(0)

### Concat all jobs together

In [96]:
import time, glob

#Iterate through list of filenames and read each in
outfilename = 'all_' + str((int(time.time()))) + ".csv"

filenames = glob.glob('*.csv')

import shutil

with open(outfilename, 'wb') as outfile:
    for filename in glob.glob('*.csv'):
        if filename == outfilename:
            # don't want to copy the output into the output
            continue
        with open(filename, 'rb') as readfile:
            shutil.copyfileobj(readfile, outfile)


In [55]:
outfile

<_io.BufferedWriter name='all_1589642043.csv'>

In [2]:
df = pd.read_csv('all_1589665998.csv', encoding='latin-1')

In [3]:
df.head()

,Unnamed: 0,Title,Company,Location,Link,Review,Salary,Description
0,0.0,Data Scientist,Lockheed Martin,"Arlington, VA",https://www.indeed.com/pagead/clk?mo=r&ad=-6NY...,4,None,Description:Are you ready to take your career ...
1,1.0,Support Scientist Ocean Data Assimilation (HAFS),IMSG/NOAA,"Arlington, VA",https://www.indeed.com/pagead/clk?mo=r&ad=-6NY...,None,None,"Overview I.M. Systems Group, Inc. (IMSG) is se..."
2,2.0,"2021048 Data Scientist $215,000.00",B4CORP,"Arlington, VA",https://www.indeed.com/pagead/clk?mo=r&ad=-6NY...,None,"Up to $215,000 a year",Position Requires a Top Secret (TS/SCI) Cleara...
3,3.0,Senior Data Engineer - Data Quality and Monito...,Freddie Mac,"Arlington, VA",https://www.indeed.com/pagead/clk?mo=r&ad=-6NY...,3.8,None,Are you passionate to work with the most valua...
4,4.0,Chief Scientist Machine Learning,Leidos,"Arlington, VA",https://www.indeed.com/pagead/clk?mo=r&ad=-6NY...,3.7,None,Description\rJob Description:\rThe Leidos Arti...


In [4]:
df = df.drop('Unnamed: 0', axis=1)

In [5]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 12155 entries, 0 to 12154
Data columns (total 7 columns):
Title          12155 non-null object
Company        12151 non-null object
Location       12155 non-null object
Link           12155 non-null object
Review         12155 non-null object
Salary         12155 non-null object
Description    12155 non-null object
dtypes: object(7)
memory usage: 664.9+ KB


#### Remove Duplicates

In [6]:
df2 = df.drop_duplicates(subset='Link', keep = 'first', inplace = False)

In [7]:
df2.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 7066 entries, 0 to 12034
Data columns (total 7 columns):
Title          7066 non-null object
Company        7064 non-null object
Location       7066 non-null object
Link           7066 non-null object
Review         7066 non-null object
Salary         7066 non-null object
Description    7066 non-null object
dtypes: object(7)
memory usage: 441.6+ KB


In [8]:
df2.groupby('Location').count().sort_values(by='Salary', ascending=True);

In [9]:
def rename_location(x):
    if "New York" in x:
        y = "New York, NY"
    else: 
        y = x
    return y

In [10]:
df2['Location2'] = df2['Location'].apply(rename_location)

/Applications/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


In [11]:
df2.to_csv("job_data_1.csv")

In [12]:
df2 = pd.read_csv("job_data_1.csv")

In [13]:
df2.Salary.unique()

array(['None', 'Up to $215,000 a year', '$60,000 a year',
       '$20 - $25 an hour', '$70,000 - $85,000 a year',
       '$45 - $55 an hour', '$102,663 - $157,709 a year',
       '$40 - $60 an hour', '$21.93 - $26.43 an hour',
       '$40,000 - $60,000 a year', '$82,326 - $105,339 a year',
       '$45 an hour', '$1,200 - $4,200 a month',
       '$100,000 - $150,000 a year', '$140,000 - $175,000 a year',
       '$67,968 - $126,062 a year', '$75 - $100 an hour',
       '$121,316 - $157,709 a year', '$72,030 - $137,012 a year',
       'Up to $125,000 a year', '$1,000 a month', '$130,000 a year',
       '$27.17 - $36.60 an hour', '$120,000 - $170,000 a year',
       'Up to $100,000 a year', '$120,000 - $150,000 a year',
       '$40,000 - $50,000 a year', '$190,000 a year', '$20 - $27 an hour',
       '$50 - $55 an hour', '$100,000 - $145,000 a year',
       '$160,000 - $175,000 a year', '$70,000 - $80,000 a year',
       '$85,000 - $165,000 a year', '$145,000 - $165,000 a year',
       '$1

#### Subset dataframe with Salary info

In [14]:
salary_df = df2[df2['Salary']!= 'None'].drop(['Location','Unnamed: 0'], axis = 1)

In [15]:
salary_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1162 entries, 2 to 7261
Data columns (total 7 columns):
Title          981 non-null object
Company        964 non-null object
Link           964 non-null object
Review         964 non-null object
Salary         964 non-null object
Description    964 non-null object
Location2      959 non-null object
dtypes: object(7)
memory usage: 72.6+ KB


We have 964 jobs with salary info. Let's look at their titles.

In [16]:
salary_list = salary_df['Salary'].unique().tolist()

##### Adjust Hourly and Daily Salary to Yearly

In [17]:
def salary_base(x):
    y=""
    if "year" in str(x):
        y = "Yearly"
    if "hour" in str(x):
        y = "Hourly"
    if "day" in str(x):
        y = "Daily"
    elif "month" in str(x):
        y = "Monthly"
    
    return y
        

In [18]:
salary_df['Rate_by'] = salary_df['Salary'].apply(salary_base)

In [19]:
salary_df = salary_df[salary_df['Rate_by']!=""]

In [20]:
indexNames = salary_df[salary_df['Rate_by']==""].index
 
# Delete these row indexes from dataFrame
salary_df.drop(indexNames , inplace=True)

In [21]:
salary_df.reset_index(drop=True);

In [22]:
salary_df.groupby('Rate_by').Salary.count()

Rate_by
Daily        1
Hourly     294
Monthly     31
Yearly     637
Name: Salary, dtype: int64

#### Split the cell and use regular expression to identify only numbers.

In [23]:
a = "From $69,415 a year" 


pattern = '\$[0-9]*[.,]?[0-9]*'
p = re.compile(pattern)
digits = p.findall(a)
digits

['$69,415']

In [24]:
salary_list;

In [25]:
salary_df[['low','high']] = salary_df.Salary.str.split('-', expand=True)

In [26]:
salary_df[['Rate_by','low','high']]

,Rate_by,low,high
2,Yearly,"Up to $215,000 a year",None
6,Yearly,"$60,000 a year",None
8,Hourly,$20,$25 an hour
11,Yearly,"$70,000","$85,000 a year"
16,Hourly,$45,$55 an hour
...,...,...,...
7192,Yearly,"$108,000","$118,000 a year"
7199,Yearly,"$89,000","$99,000 a year"
7208,Yearly,"$89,000","$99,000 a year"
7212,Yearly,"$80,000","$90,000 a year"


In [27]:
def find_salary(a):
    if a:
        pattern = '\$[0-9]*[.,]?[0-9]*'
        p = re.compile(pattern)
        digits = p.findall(a)
        new1 = digits[0].replace("$","")
        new2 = new1.replace(",","")
        
        return float(new2)
    else:
        return 0

In [28]:
salary_df['min'] = salary_df['low'].apply(find_salary)

In [29]:
salary_df['max'] = salary_df['high'].apply(find_salary)

In [30]:
salary_df.head();

In [31]:
salary_df.to_csv("salary_df_5.20.csv")

#### Calculate the adjusted salary (turn all daily/hourly wage to yearly), use average

In [32]:
#Calculate average salary
def adjust_salary(a, b):
    adjusted_salary=[]
    for x, y in zip(a,b):
        if y > 0:
            avg_salary = (x+y)/2
        
        else:
            avg_salary = x
            
        adjusted_salary.append(avg_salary)
        
    return adjusted_salary               

In [33]:
salary_df['adjusted_salary'] = adjust_salary(salary_df['min'], salary_df['max'])

In [34]:
#turn hourly, daily and monthly wage to yearly 
def adjust_salary2(m,n):
    adjusted_yearly_salary = []
    for x, y in zip(m,n):
        if x == "Yearly":
            yearly_salary = y
        if x == "Monthly":
            yearly_salary = y*12
        if x == "Hourly":
            yearly_salary = y*40*52
        if x == "Daily":
            yearly_salary = y*5*52
            
        adjusted_yearly_salary.append(yearly_salary)
        
    return adjusted_yearly_salary
        

In [35]:
salary_df['adjusted_yearly_salary'] = adjust_salary2(salary_df['Rate_by'], salary_df['adjusted_salary'])

In [36]:
salary_df.head()

,Title,Company,Link,Review,Salary,Description,Location2,Rate_by,low,high,min,max,adjusted_salary,adjusted_yearly_salary
2,"2021048 Data Scientist $215,000.00",B4CORP,https://www.indeed.com/pagead/clk?mo=r&ad=-6NY...,None,"Up to $215,000 a year",Position Requires a Top Secret (TS/SCI) Cleara...,"Arlington, VA",Yearly,"Up to $215,000 a year",None,215000.0,0.0,215000.0,215000.0
6,Data Science/Data Analyst,CRAiC LLC,https://www.indeed.com/company/CRAiC-LLC/jobs/...,None,"$60,000 a year",Cost Readiness Analytics (CRAiC LLC) is lookin...,"Arlington, VA",Yearly,"$60,000 a year",None,60000.0,0.0,60000.0,60000.0
8,Data Science Intern,MilliporeSigma,https://www.indeed.com/company/Serono/jobs/Dat...,3.9,$20 - $25 an hour,Job Title: Data Science Intern\rA career with ...,"Arlington, VA",Hourly,$20,$25 an hour,20.0,25.0,22.5,46800.0
11,Jr. Data Scientist,Numero Data LLC,https://www.indeed.com/company/Numero-Data-LLC...,None,"$70,000 - $85,000 a year",Ideal candidate should have a degree in a quan...,"Arlington, VA",Yearly,"$70,000","$85,000 a year",70000.0,85000.0,77500.0,77500.0
16,Business Data Quality Engineer,HRU Technical Resources,https://www.indeed.com/pagead/clk?mo=r&ad=-6NY...,None,$45 - $55 an hour,Please note that this is not a corp to corp or...,"Arlington, VA",Hourly,$45,$55 an hour,45.0,55.0,50.0,104000.0


#### Join with Cost of Living Index

In [37]:
COI = pd.read_excel("CostofLiving2020.xlsx")

In [38]:
COI.head()

,Rank,City,Cost of Living Index,Rent Index,Cost of Living Plus Rent Index,Groceries Index,Restaurant Price Index,Local Purchasing Power Index
0,1,"New York, NY",100.00,100.00,100.00,100.00,100.00,100.00
1,2,"San Francisco, CA",91.51,115.58,103.02,86.91,94.45,135.90
2,3,"Anchorage, AK",91.36,38.94,66.29,88.42,78.89,118.44
3,4,"Honolulu, HI",89.70,63.03,76.94,89.83,83.60,88.29
4,5,"Brooklyn, NY",88.82,80.49,84.84,83.22,92.77,92.95


In [ ]:
salary_df2 = salary_df.join(COI)

### Helper Functions to deal with text data

* Do we remove stop words or not?
* Do we stem or lemmatize our text data, or leave the words as is?
* Is basic tokenization enough, or do we need to support special edge cases through the use of regex?
* Do we use the entire vocabulary, or just limit the model to a subset of the most frequently used words? If so, how many?
* Do we engineer other features, such as bigrams, or POS tags, or Mutual Information Scores?
* What sort of vectorization should we use in our model? Boolean Vectorization? Count Vectorization? TF-IDF? More advanced vectorization strategies such as Word2Vec?

In [2]:
salary_df = pd.read_csv("salary_final.csv")

In [3]:
salary_df.head()

,Unnamed: 0,Title,Company,Link,Review,Salary,Description,Location2,Rate_by,low,high,min,max,adjusted_salary,adjusted_yearly_salary
0,2,"2021048 Data Scientist $215,000.00",B4CORP,https://www.indeed.com/pagead/clk?mo=r&ad=-6NY...,None,"Up to $215,000 a year",Position Requires a Top Secret (TS/SCI) Cleara...,"Arlington, VA",Yearly,"Up to $215,000 a year",NaN,215000.0,0.0,215000.0,215000.0
1,6,Data Science/Data Analyst,CRAiC LLC,https://www.indeed.com/company/CRAiC-LLC/jobs/...,None,"$60,000 a year",Cost Readiness Analytics (CRAiC LLC) is lookin...,"Arlington, VA",Yearly,"$60,000 a year",NaN,60000.0,0.0,60000.0,60000.0
2,8,Data Science Intern,MilliporeSigma,https://www.indeed.com/company/Serono/jobs/Dat...,3.9,$20 - $25 an hour,Job Title: Data Science Intern\rA career with ...,"Arlington, VA",Hourly,$20,$25 an hour,20.0,25.0,22.5,46800.0
3,11,Jr. Data Scientist,Numero Data LLC,https://www.indeed.com/company/Numero-Data-LLC...,None,"$70,000 - $85,000 a year",Ideal candidate should have a degree in a quan...,"Arlington, VA",Yearly,"$70,000","$85,000 a year",70000.0,85000.0,77500.0,77500.0
4,16,Business Data Quality Engineer,HRU Technical Resources,https://www.indeed.com/pagead/clk?mo=r&ad=-6NY...,None,$45 - $55 an hour,Please note that this is not a corp to corp or...,"Arlington, VA",Hourly,$45,$55 an hour,45.0,55.0,50.0,104000.0


In [4]:
def clean_word(job_descriptions):
    jd_data=[]
    pattern = "([a-zA-Z]+(?:'[a-z]+)?)"
    stopwords_list = stopwords.words('english')
    stopwords_list += list(string.punctuation)
    stopwords_list += ['0', '1', '2', '3', '4', '5', '6', '7', '8', '9']
    
    for jd in job_descriptions:
        jd_tokens_raw = nltk.regexp_tokenize(jd, pattern)
        jd_tokens=[word.lower() for word in jd_tokens_raw]
        jd_words_stopped = [word for word in jd_tokens if word not in stopwords_list]
        jd_data.append(jd_words_stopped)
    return pd.Series(jd_data) # Turn list of lists to series of lists to use in Word2Vec

In [5]:
#Combine Title with the Description
salary_df['combined_text'] = salary_df['Title'].str.cat(salary_df['Description'], sep = ' ')